In [82]:
import pandas as pd
import numpy as np
from pandas import read_csv
from pmdarima.arima import auto_arima
from datetime import datetime
import matplotlib.pyplot as plt
import tsfresh
from pmdarima.arima import ADFTest
from tsfresh import extract_features
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

In [83]:
values = read_csv("../../Time_series_data.csv")

In [84]:
values.head(13)

,Month,Sales
0,2013-01,2815
1,2013-02,2672
2,2013-03,2755
3,2013-04,2721
4,2013-05,2946
5,2013-06,3036
6,2013-07,2282
7,2013-08,2212
8,2013-09,2922
9,2013-10,4301


In [85]:

values = values.rename(columns={values.columns[1]: 'Data'})
values = values.rename(columns={values.columns[0]: 'Time'})
values['Time'] = pd.to_datetime(
    values['Time'], errors='coerce')

In [86]:
freq = pd.infer_freq(values["Time"])

In [87]:
# end_date = pd.to_datetime(x) + pd.DateOffset(months=10)
# print(end_date)

In [88]:
values.set_index('Time', inplace=True)

In [89]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(
    values, test_size=0.2, shuffle=False)

rnn_train = train
rnn_test = test

In [90]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaler.fit(rnn_train)
scaled_train = scaler.transform(rnn_train)
scaled_test = scaler.transform(rnn_test)

In [91]:
from keras.preprocessing.sequence import TimeseriesGenerator

# define generator
n_input = 12
n_features = 1
generator = TimeseriesGenerator(
    scaled_train, scaled_train, length=n_input, batch_size=1)

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [92]:
# define model
model = Sequential()
model.add(LSTM(100, activation='relu',
            input_shape=(n_input, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.summary()
model.fit(generator, epochs=50)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 100)               40800     
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 40901 (159.77 KB)
Trainable params: 40901 (159.77 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/50
72/72 [==============================] - 2s 4ms/step - loss: 0.0589
Epoch 2/50
72/72 [==============================] - 0s 4ms/step - loss: 0.0442
Epoch 3/50
72/72 [==============================] - 0s 4ms/step - loss: 0.0426
Epoch 4/50
72/72 [==============================] - 0s 4ms/step - loss: 0.0356
Epoch 5/50
72/72 [==============================] - 0s 4ms/step - loss: 0.0244
Epoch 6/50
72/72 [==============================] - 0s 4ms/step - loss: 0.0252
Epoch 7/50
72/72 [==============================] - 0s 4ms/step - loss: 0.0416
Epoch 8/50
72/72 [==============================] - 0s 4ms/step - loss: 0.0256
Epoch 9/50
72/72 [==============================] - 0s 4ms/step - loss: 0.0142
Epoch 10/50
72/72 [==============================] - 0s 4ms/step - loss: 0.0103
Epoch 11/50
72/72 [==============================] - 0s 4ms/step - loss: 0.0104
Epoch 12/50
72/72 [==============================] - 0s 4ms/step - loss: 0.0129
Epoch 13/50
72/72 [==============================

In [93]:
last_train_batch = scaled_train[-12:]
last_train_batch = last_train_batch.reshape((1, n_input, n_features))


In [94]:
last_train_batch

array([[[0.08636474],
        [0.10742931],
        [0.14558859],
        [0.1755651 ],
        [0.1096978 ],
        [0.19549542],
        [0.21421048],
        [0.0133679 ],
        [0.29555213],
        [0.39301628],
        [0.66993438],
        [0.93194523]]])

In [95]:
model.predict(last_train_batch)

test_predictions = []

first_eval_batch = scaled_train[-n_input:]
current_batch = first_eval_batch.reshape((1, n_input, n_features))

1/1 [==============================] - 0s 188ms/step


In [96]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 100)               40800     
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 40901 (159.77 KB)
Trainable params: 40901 (159.77 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [97]:
for i in range(len(rnn_test)):
    # get the prediction value for the first batch
    current_pred = model.predict(current_batch)[0]

    # append the prediction into the array
    test_predictions.append(current_pred)
    
    # use the prediction to update the batch and remove the first value
    current_batch = np.append(current_batch[:, 1:, :], [
                                [current_pred]], axis=1)

1/1 [==============================] - 0s 24ms/step


In [98]:
current_batch

array([[[0.48265636],
        [0.69599384],
        [0.83337021],
        [0.2187189 ],
        [0.20008415],
        [0.20542215],
        [0.21479109],
        [0.20265721],
        [0.24025083],
        [0.26893696],
        [0.22624701],
        [0.41539305]]])

In [99]:
true_predictions = scaler.inverse_transform(test_predictions)
rnn_test['Predictions'] = true_predictions

In [100]:
forecast_errors = [rnn_test['Data'][i]-rnn_test['Predictions'][i] for i in range(len(rnn_test))]

In [101]:
forecast_errors

[179.9218740016222,
 -517.9855819344521,
 425.45999559760094,
 629.2033453434706,
 1321.5106441825628,
 577.4984444379807,
 75.96219836175442,
 -1800.039199128747,
 -22.753655791282654,
 -549.42744743824,
 -312.6519756913185,
 810.7115135192871,
 75.35260154306889,
 -478.63866144418716,
 468.4744258970022,
 563.8335848152637,
 543.6020835489035,
 773.5840561389923,
 -594.4889205992222,
 -2952.566876709461,
 -823.1964715123177]

In [102]:
mean_forecast_error = np.mean(forecast_errors)

In [103]:
mae = mean_absolute_error(rnn_test['Data'], rnn_test['Predictions'])
mae

690.3268360779399

In [106]:
mse = mean_squared_error(rnn_test['Data'], rnn_test['Predictions'])
mse

889314.2211560281

In [104]:
mape = mean_absolute_percentage_error(rnn_test['Data'], rnn_test['Predictions'])
mape

0.2432546575620112

In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Tạo một DataFrame giả định
data = pd.DataFrame({
    'Age': [25, 30, 22, 35, 28],
    'Gender': ['Male', 'Female', 'Male', 'Female', 'Male'],
    'Income': [50000, 60000, 45000, 70000, 55000]
})

# Data Preprocessing - Chuẩn hóa dữ liệu
scaler = StandardScaler()
data['Income'] = scaler.fit_transform(data[['Income']])

# Data Preprocessing - Mã hóa biến phân loại
encoder = LabelEncoder()
data['Gender'] = encoder.fit_transform(data['Gender'])

# Data Preparation - Tạo tập huấn luyện và kiểm tra
train_data = data.iloc[:3]  # Dữ liệu huấn luyện
test_data = data.iloc[3:]   # Dữ liệu kiểm tra

print("Preprocessed Data:")
print(data)
print("Training Data:")
print(train_data)
print("Testing Data:")
print(test_data)

Preprocessed Data:
   Age  Gender    Income
0   25       1 -0.697486
1   30       0  0.464991
2   22       1 -1.278724
3   35       0  1.627467
4   28       1 -0.116248
Training Data:
   Age  Gender    Income
0   25       1 -0.697486
1   30       0  0.464991
2   22       1 -1.278724
Testing Data:
   Age  Gender    Income
3   35       0  1.627467
4   28       1 -0.116248
